In [8]:
import numpy as np
import random
import struct
import matplotlib.pyplot as plt
import os
import time
import pickle
from scipy.stats import mannwhitneyu
from scipy.io import loadmat
from typing import Union, List, Tuple, Callable, Dict, Any

In [9]:
"""
****************************************GNBG****************************************
Author: Danial Yazdani
Last Edited: March 09, 2024
Title: Generalized Numerical Benchmark Generator (GNBG)
--------
Description: 
This Python code implements a set of 24 problem instances generated by the 
Generalized Numerical Benchmark Generator (GNBG). 
The GNBG parameter setting for each problem instance is stored in separate '.mat' files
for ease of access and execution. The code is designed to work directly with 
these pre-configured instances. Note that the code operates independently of the GNBG
generator file and only uses the saved configurations of the instances.
--------
Reference: 
D. Yazdani, M. N. Omidvar, D. Yazdani, K. Deb, and A. H. Gandomi, "GNBG: A Generalized
and Configurable Benchmark Generator for Continuous Numerical Optimization," 
arXiv prepring	arXiv:2312.07083, 2023.

AND

A. H. Gandomi, D. Yazdani, M. N. Omidvar, and K. Deb, "GNBG-Generated Test Suite for 
Box-Constrained Numerical Global Optimization," arXiv preprint arXiv:2312.07034, 2023.

If you are using GNBG and this code in your work, you should cite the references provided above.    
--------
License:
This program is to be used under the terms of the GNU General Public License
(http://www.gnu.org/copyleft/gpl.html).
Author: Danial Yazdani
e-mail: danial DOT yazdani AT gmail DOT com
Copyright notice: (c) 2023 Danial Yazdani
**************************************************************************
"""

class GNBG:
    def __init__(self, MaxEvals, AcceptanceThreshold, Dimension, CompNum, MinCoordinate, MaxCoordinate, CompMinPos, CompSigma, CompH, Mu, Omega, Lambda, RotationMatrix, OptimumValue, OptimumPosition):
        self.MaxEvals = int(MaxEvals)
        self.AcceptanceThreshold = AcceptanceThreshold
        self.Dimension = int(Dimension)
        self.CompNum = int(CompNum)
        self.MinCoordinate = MinCoordinate
        self.MaxCoordinate = MaxCoordinate
        self.CompMinPos = CompMinPos
        self.CompSigma = CompSigma
        self.CompH = CompH
        self.Mu = Mu
        self.Omega = Omega
        self.Lambda = Lambda
        self.RotationMatrix = RotationMatrix
        self.OptimumValue = OptimumValue
        self.OptimumPosition = OptimumPosition
        self.FEhistory = []
        self.FE = 0
        self.AcceptanceReachPoint = np.inf
        self.BestFoundResult = np.inf

    def evaluate(self, X_list: list) -> float:
        X = np.array(X_list).reshape(1, -1)

        if X.shape[1] != self.Dimension:
             return float('inf')

        if len(X.shape)<2:
            X = X.reshape(1,-1)
        SolutionNumber = X.shape[0]
        result = np.nan * np.ones(SolutionNumber)
        for jj in range(SolutionNumber):
            x = X[jj, :].reshape(-1, 1)
            x = np.clip(x, self.MinCoordinate, self.MaxCoordinate)

            f = np.nan * np.ones(self.CompNum)
            for k in range(self.CompNum):
                if len(self.RotationMatrix.shape) == 3:
                    if k >= self.RotationMatrix.shape[2]:
                         f[k] = float('inf')
                         continue
                    rotation_matrix = self.RotationMatrix[:, :, k]
                else:
                    rotation_matrix = self.RotationMatrix

                term1 = (x - self.CompMinPos[k, :].reshape(-1, 1)).T
                term2 = rotation_matrix.T
                if term1.shape[1] != term2.shape[0]:
                    f[k] = float('inf')
                    continue

                a = self.transform(term1 @ term2, self.Mu[k, :], self.Omega[k, :])

                term3 = rotation_matrix
                term4 = (x - self.CompMinPos[k, :].reshape(-1, 1))
                if term3.shape[1] != term4.shape[0]:
                      f[k] = float('inf')
                      continue

                b = self.transform(term3 @ term4, self.Mu[k, :], self.Omega[k, :])

                if a.shape[1] != self.CompH[k, :].shape[0] or self.CompH[k, :].shape[0] != b.shape[0]:
                     f[k] = float('inf')
                     continue


                try:
                    base = a @ np.diag(self.CompH[k, :]) @ b
                    lambda_val = self.Lambda[k, 0] if self.Lambda.ndim > 1 else self.Lambda[k]

                    if base < 0 and lambda_val % 1 != 0:
                         term_result = float('inf')
                    elif np.isclose(base, 0) and lambda_val <= 0:
                         term_result = float('inf')
                    else:
                         term_result = (base ** lambda_val).item()
                         if not np.isfinite(term_result):
                              term_result = float('inf')

                    f[k] = self.CompSigma[k, 0].item() + term_result
                except Exception as calc_e:
                     f[k] = float('inf')


            finite_f = f[np.isfinite(f)]
            if finite_f.size > 0:
                 result[jj] = np.min(finite_f)
            else:
                 result[jj] = float('inf')


        return result[0]

    def transform(self, X, Alpha, Beta):
        Y = X.copy().astype(float)
        try:
             tmp_pos = (X > 1e-12)
             if np.any(tmp_pos):
                  log_X_pos = np.log(X[tmp_pos])
                  sin_term_pos = np.sin(Beta[0] * log_X_pos) + np.sin(Beta[1] * log_X_pos)
                  Y[tmp_pos] = np.exp(log_X_pos + Alpha[0] * sin_term_pos)

             tmp_neg = (X < -1e-12)
             if np.any(tmp_neg):
                  log_X_neg = np.log(-X[tmp_neg])
                  sin_term_neg = np.sin(Beta[2] * log_X_neg) + np.sin(Beta[3] * log_X_neg)
                  Y[tmp_neg] = -np.exp(log_X_neg + Alpha[1] * sin_term_neg)

             tmp_zero = np.logical_not(np.logical_or(tmp_pos, tmp_neg))
             Y[tmp_zero] = 0.0

             if not np.all(np.isfinite(Y)):
                  Y = np.clip(Y, -1e100, 1e100)

        except Exception as e:
             return np.zeros_like(X)

        return Y

def load_instance(problem_index, folder_path):
    filename = f"f{problem_index}.mat"
    filepath = os.path.join(folder_path, filename)
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Soubor {filename} nebyl nalezen v {folder_path}")

    try:
        GNBG_tmp = loadmat(filepath)['GNBG']
        MaxEvals = np.array(GNBG_tmp['MaxEvals'][0,0])[0,0]
        AcceptanceThreshold = np.array(GNBG_tmp['AcceptanceThreshold'][0,0])[0,0]
        Dimension = np.array(GNBG_tmp['Dimension'][0,0])[0,0]
        CompNum = np.array(GNBG_tmp['o'][0,0])[0,0] # 'o' je počet komponent
        MinCoordinate = np.array(GNBG_tmp['MinCoordinate'][0,0])[0,0]
        MaxCoordinate = np.array(GNBG_tmp['MaxCoordinate'][0,0])[0,0]
        CompMinPos = np.array(GNBG_tmp['Component_MinimumPosition'][0, 0])
        CompSigma = np.array(GNBG_tmp['ComponentSigma'][0, 0], dtype=np.float64)
        CompH = np.array(GNBG_tmp['Component_H'][0, 0])
        Mu = np.array(GNBG_tmp['Mu'][0, 0])
        Omega = np.array(GNBG_tmp['Omega'][0, 0])
        Lambda = np.array(GNBG_tmp['lambda'][0, 0])
        RotationMatrix = np.array(GNBG_tmp['RotationMatrix'][0, 0])
        OptimumValue = np.array(GNBG_tmp['OptimumValue'][0,0])[0,0]
        OptimumPosition = np.array(GNBG_tmp['OptimumPosition'][0, 0])

        instance_name = f"f{problem_index}"

        gnbg_obj = GNBG(
            MaxEvals, AcceptanceThreshold, Dimension, CompNum, MinCoordinate,
            MaxCoordinate, CompMinPos, CompSigma, CompH, Mu, Omega, Lambda,
            RotationMatrix, OptimumValue, OptimumPosition
        )
        setattr(gnbg_obj, 'Name', instance_name)
        return gnbg_obj

    except Exception as e:
        print(f"Chyba při načítání instance f{problem_index} ze souboru {filename}: {e}")
        return None



In [10]:
POPULATION_SIZE = 100
MUTATION_RATE = 0.01
CROSSOVER_RATE = 0.8
SELECTION_PRESSURE = 2
RUNS = 31
BITS_PER_DIM_BINARY = 32

def real_to_binary_fixed_point(value: float, min_val: float, max_val: float, num_bits: int) -> str:
    if min_val >= max_val:
        if np.isclose(min_val, max_val):
             return '0' * num_bits 
        raise ValueError("min_val musí být menší než max_val")
    if num_bits <= 0:
        raise ValueError("num_bits musí být kladné číslo")

    value = max(min(value, max_val), min_val)

    range_size = max_val - min_val
    if np.isclose(range_size, 0):
         normalized = 0.0
    else:
         normalized = (value - min_val) / range_size

    scale = (2 ** num_bits) - 1
    integer = int(round(normalized * scale))
    integer = max(0, min(integer, scale))

    return bin(integer)[2:].zfill(num_bits)

def binary_to_real_fixed_point(binary_str: str, min_val: float, max_val: float) -> float:
    if not binary_str or not all(c in "01" for c in binary_str):

         return min_val
    try:
        decimal_value = int(binary_str, 2)
    except ValueError:
         return min_val

    num_bits = len(binary_str)
    max_binary = (2 ** num_bits) - 1
    if max_binary <= 0:
         return min_val

    real_value = min_val + (decimal_value / max_binary) * (max_val - min_val)
    return max(min_val, min(real_value, max_val))


def float_to_ieee754(value: float) -> str:
    if not isinstance(value, (float, int)):
        try: value = float(value)
        except (ValueError, TypeError): value = 0.0

    if not np.isfinite(value):
         value = np.clip(value, np.finfo(np.float32).min, np.finfo(np.float32).max)
         if not np.isfinite(value): value = 0.0

    try:
        packed = struct.pack('!f', float(value))
    except OverflowError:
        packed = struct.pack('!f', np.finfo(np.float32).max if float(value) > 0 else np.finfo(np.float32).min)
    except Exception:
         packed = struct.pack('!f', 0.0)

    return ''.join(f'{byte:08b}' for byte in packed)

def binary_to_real_ieee754(binary_str: str) -> float:
    if len(binary_str) != 32: return 0.0
    try:
        int_val = int(binary_str, 2)
        if not (0 <= int_val <= 0xFFFFFFFF):
            return 0.0
        bytes_obj = int_val.to_bytes(4, byteorder='big')
        value = struct.unpack('!f', bytes_obj)[0]
    except (OverflowError, struct.error, ValueError):
        return 0.0
    if not np.isfinite(value): return 0.0
    return value

def gray_code_encode(binary_str):
    if not all(c in "01" for c in binary_str): return '0' * len(binary_str) # Fallback
    try:
        n = int(binary_str, 2)
        gray = n ^ (n >> 1)
        return bin(gray)[2:].zfill(len(binary_str))
    except ValueError: return '0' * len(binary_str)

def gray_code_decode(gray_str):
    if not all(c in "01" for c in gray_str): return '0' * len(gray_str) # Fallback
    try:
        gray = int(gray_str, 2)
        binary = gray
        mask = gray >> 1
        while mask != 0:
            binary ^= mask
            mask >>= 1
        return bin(binary)[2:].zfill(len(gray_str))
    except ValueError: return '0' * len(gray_str)


class GeneticAlgorithmBase:
    def __init__(self,
                 population_size: int,
                 dimension: int,
                 value_range: Tuple[float, float],
                 mutation_rate: float,
                 crossover_rate: float,
                 selection_pressure: int):
        if population_size <= 0 or population_size % 2 != 0:
            raise ValueError("Population size musí být kladné sudé číslo")
        if not 0 <= mutation_rate <= 1:
            raise ValueError("Mutation rate musí být v rozsahu [0, 1]")
        if not 0 <= crossover_rate <= 1:
            raise ValueError("Crossover rate musí být v rozsahu [0, 1]")
        if selection_pressure < 2:
            raise ValueError("Selection pressure musí být >= 2")

        self.population_size = population_size
        self.dimension = dimension
        self.value_range = value_range
        self.min_val, self.max_val = value_range
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.selection_pressure = selection_pressure
        self.population = None
        self.fitness_func_wrapper = None 

    def set_fitness_function(self, gnbg_instance: GNBG):
        self.fitness_func_wrapper = lambda individual_values: gnbg_instance.evaluate(individual_values)

    def evaluate_fitness(self, individual: Any) -> float:
        if self.fitness_func_wrapper is None:
            raise RuntimeError("Fitness funkce (wrapper) není nastavena.")
        try:
            values = self._decode_individual(individual)
            if not all(np.isfinite(v) for v in values): return float('inf')

            if len(values) != self.dimension: return float('inf')

            fitness = self.fitness_func_wrapper(values)
            if not np.isfinite(fitness): return float('inf')
            return fitness

        except Exception as e:
            return float('inf')

    def _tournament_selection(self) -> List[Any]:
         selected = []
         population_fitness = [(ind, self.evaluate_fitness(ind)) for ind in self.population]
         valid_population_fitness = [(ind, fit) for ind, fit in population_fitness if np.isfinite(fit)]

         if not valid_population_fitness:
              return list(self.population)

         for _ in range(self.population_size):
              tournament_candidates = random.sample(valid_population_fitness, min(self.selection_pressure, len(valid_population_fitness)))
              winner_individual, _ = min(tournament_candidates, key=lambda item: item[1])
              selected.append(winner_individual)
         return selected

    def _initialize_population(self): raise NotImplementedError
    def _decode_individual(self, individual: Any) -> List[float]: raise NotImplementedError
    def _encode_values(self, values: List[float]) -> Any: raise NotImplementedError
    def _crossover(self, parents: List[Any]) -> List[Any]: raise NotImplementedError
    def _mutate(self, population: List[Any]) -> List[Any]: raise NotImplementedError
    def evolve(self) -> Tuple[Any, float]: raise NotImplementedError

class BinaryGeneticAlgorithm(GeneticAlgorithmBase):
    def __init__(self, representation: str, bits_per_dim: int, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.representation = representation
        self.bits_per_dim = self._validate_representation(representation, bits_per_dim)
        if self.dimension <= 0: raise ValueError("Dimenze musí být > 0")
        self.chromosome_length = self.dimension * self.bits_per_dim
        if self.chromosome_length <= 0: raise ValueError("Délka chromozomu musí být > 0")
        self.population = self._initialize_population()

    def _validate_representation(self, rep: str, bits: int) -> int:
        if rep == 'ieee754': return 32
        elif rep in ['fixed-point', 'gray']:
            if bits <= 0: raise ValueError("Počet bitů musí být > 0")
            return bits
        else: raise ValueError(f"Nepodporovaná reprezentace: {rep}")

    def _initialize_population(self) -> List[str]:
        population = []
        attempts = 0
        max_attempts = self.population_size * 5 

        while len(population) < self.population_size and attempts < max_attempts:
             values = [random.uniform(self.min_val, self.max_val) for _ in range(self.dimension)]
             try:
                  encoded_individual = self._encode_values(values)
                  if len(encoded_individual) == self.chromosome_length:
                       population.append(encoded_individual)
             except Exception as e:
                  pass 
             attempts += 1

        if len(population) < self.population_size:
             raise RuntimeError(f"Nepodařilo se inicializovat dostatek validních jedinců ({len(population)}/{self.population_size}). Zkontrolujte _encode_values.")

        return population


    def _decode_individual(self, individual: str) -> List[float]:
        if len(individual) != self.chromosome_length:
            return [random.uniform(self.min_val, self.max_val) for _ in range(self.dimension)]

        values = []
        for i in range(self.dimension):
            start_idx = i * self.bits_per_dim
            end_idx = start_idx + self.bits_per_dim
            chromosome_part = individual[start_idx:end_idx]

            try:
                if self.representation == 'fixed-point':
                    val = binary_to_real_fixed_point(chromosome_part, self.min_val, self.max_val)
                elif self.representation == 'gray':
                    binary_str = gray_code_decode(chromosome_part)
                    val = binary_to_real_fixed_point(binary_str, self.min_val, self.max_val)
                elif self.representation == 'ieee754':
                    val = binary_to_real_ieee754(chromosome_part)
                    val = np.clip(val, self.min_val, self.max_val)
                else: raise ValueError("Neplatná reprezentace")

                if not np.isfinite(val): val = random.uniform(self.min_val, self.max_val)
                values.append(val)
            except Exception: 
                 values.append(random.uniform(self.min_val, self.max_val))

        if len(values) != self.dimension:
            values = (values + [random.uniform(self.min_val, self.max_val)] * self.dimension)[:self.dimension]

        return [np.clip(v, self.min_val, self.max_val) for v in values]


    def _encode_values(self, values: List[float]) -> str:
        if len(values) != self.dimension: raise ValueError("Nesprávný počet hodnot pro kódování")
        encoded_parts = []
        for val in values:
            clipped_val = np.clip(val, self.min_val, self.max_val)
            try:
                if self.representation == 'fixed-point':
                    encoded_parts.append(real_to_binary_fixed_point(clipped_val, self.min_val, self.max_val, self.bits_per_dim))
                elif self.representation == 'gray':
                    binary_str = real_to_binary_fixed_point(clipped_val, self.min_val, self.max_val, self.bits_per_dim)
                    encoded_parts.append(gray_code_encode(binary_str))
                elif self.representation == 'ieee754':
                    encoded_parts.append(float_to_ieee754(clipped_val))
                else: raise ValueError("Neplatná reprezentace")
            except Exception:
                 encoded_parts.append('0' * self.bits_per_dim)

        encoded_string = "".join(encoded_parts)
        if len(encoded_string) != self.chromosome_length:
             raise RuntimeError(f"Chyba kódování délky: {len(encoded_string)} vs {self.chromosome_length}")
        return encoded_string

    def _crossover(self, parents: List[str]) -> List[str]:
        offspring = []
        num_parents = len(parents)
        if num_parents % 2 != 0:
            if not parents: return []
            parents.append(random.choice(parents))
            num_parents += 1

        for i in range(0, num_parents, 2):
            p1, p2 = parents[i], parents[i+1]

            if len(p1) != self.chromosome_length or len(p2) != self.chromosome_length:

                 continue 

            if random.random() < self.crossover_rate and self.chromosome_length > 1 :
                cross_point = random.randint(1, self.chromosome_length - 1)
                c1 = p1[:cross_point] + p2[cross_point:]
                c2 = p2[:cross_point] + p1[cross_point:]
                if len(c1) == self.chromosome_length and len(c2) == self.chromosome_length:
                     offspring.extend([c1, c2])
                else:

                     pass
            else:
                offspring.extend([p1, p2])

        return offspring 


    def _mutate(self, population: List[str]) -> List[str]:
        mutated_population = []
        if self.chromosome_length <= 0: return population 

        bit_mutation_prob = self.mutation_rate / self.chromosome_length

        for individual in population:
            if len(individual) == self.chromosome_length:
                mutated_individual = list(individual)
                for i in range(self.chromosome_length):
                    if random.random() < bit_mutation_prob:
                        mutated_individual[i] = '1' if individual[i] == '0' else '0'
                mutated_population.append("".join(mutated_individual))


        return mutated_population

    def evolve(self) -> Tuple[str, float]:
        population_fitness = []
        valid_population = []
        for ind in self.population:
             if len(ind) == self.chromosome_length:
                  fitness = self.evaluate_fitness(ind)
                  if np.isfinite(fitness):
                       population_fitness.append((ind, fitness))
                       valid_population.append(ind)

        if not valid_population:
            self.population = self._initialize_population()
            fitness_values = [self.evaluate_fitness(ind) for ind in self.population]
            best_idx = np.argmin(fitness_values) if fitness_values else 0
            return self.population[best_idx], fitness_values[best_idx] if fitness_values else float('inf')

        valid_population_fitness = [(ind, fit) for ind, fit in population_fitness]
        elite_individual, elite_fitness = min(valid_population_fitness, key=lambda item: item[1])


        original_pop = self.population
        self.population = valid_population 
        selected = self._tournament_selection()
        self.population = original_pop 

        offspring = self._crossover(selected)
        mutated_offspring = self._mutate(offspring)

        new_population_fitness = []
        valid_new_individuals = []
        for ind in mutated_offspring:
             if len(ind) == self.chromosome_length:
                  fitness = self.evaluate_fitness(ind)
                  if np.isfinite(fitness):
                       new_population_fitness.append((ind, fitness))
                       valid_new_individuals.append(ind)


        combined_valid_population = [elite_individual] + valid_new_individuals
        combined_valid_fitness = [elite_fitness] + [fit for _, fit in new_population_fitness]

        sorted_indices = np.argsort(combined_valid_fitness)
        next_generation = [combined_valid_population[i] for i in sorted_indices]

        needed = self.population_size - len(next_generation)
        if needed > 0:

             fillers = [ind for ind, fit in sorted(valid_population_fitness, key=lambda item: item[1]) if ind != elite_individual]
             next_generation.extend(fillers[:needed])
             needed = self.population_size - len(next_generation)

             attempts = 0
             while needed > 0 and attempts < self.population_size:
                  values = [random.uniform(self.min_val, self.max_val) for _ in range(self.dimension)]
                  try:
                       new_ind = self._encode_values(values)
                       if len(new_ind) == self.chromosome_length:
                            next_generation.append(new_ind)
                            needed -= 1
                  except Exception:
                       pass 
                  attempts +=1

        self.population = next_generation[:self.population_size]


        if not self.population: 
             self.population = self._initialize_population() 
             if not self.population: return "", float('inf') 

        best_new_fitness = self.evaluate_fitness(self.population[0])
        return self.population[0], best_new_fitness


class RealValueGeneticAlgorithm(GeneticAlgorithmBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if self.max_val == self.min_val: self.mutation_sigma = 0.1
        else: self.mutation_sigma = (self.max_val - self.min_val) * 0.1
        self.crossover_alpha = 0.5
        self.population = self._initialize_population()

    def _initialize_population(self) -> List[List[float]]:
        return [[random.uniform(self.min_val, self.max_val) for _ in range(self.dimension)]
                for _ in range(self.population_size)]

    def _decode_individual(self, individual: List[float]) -> List[float]:

        if len(individual) != self.dimension:
             individual = (individual + [random.uniform(self.min_val, self.max_val)] * self.dimension)[:self.dimension]
        return [np.clip(val, self.min_val, self.max_val) for val in individual]


    def _encode_values(self, values: List[float]) -> List[List[float]]:
        if len(values) != self.dimension:
             raise ValueError(f"Nesprávný počet hodnot pro kódování RVGA: {len(values)} vs {self.dimension}")
        return [np.clip(val, self.min_val, self.max_val) for val in values]


    def _crossover(self, parents: List[List[float]]) -> List[List[float]]:
        offspring = []
        num_parents = len(parents)
        if num_parents % 2 != 0:
             if not parents: return []
             parents.append(random.choice(parents))
             num_parents += 1

        for i in range(0, num_parents, 2):
            p1, p2 = parents[i], parents[i+1]

            if len(p1) != self.dimension or len(p2) != self.dimension:
                 continue 

            if random.random() < self.crossover_rate:
                alpha = self.crossover_alpha
                c1 = [alpha * x + (1 - alpha) * y for x, y in zip(p1, p2)]
                c2 = [(1 - alpha) * x + alpha * y for x, y in zip(p1, p2)]
                c1 = [np.clip(val, self.min_val, self.max_val) for val in c1]
                c2 = [np.clip(val, self.min_val, self.max_val) for val in c2]
                if len(c1) == self.dimension and len(c2) == self.dimension:
                     offspring.extend([c1, c2])
            else:
                offspring.extend([p1, p2])
        return offspring


    def _mutate(self, population: List[List[float]]) -> List[List[float]]:
        mutated_population = []
        for individual in population:
            if len(individual) == self.dimension:
                mutated_individual = list(individual)
                for i in range(self.dimension):
                    if random.random() < self.mutation_rate:
                        mutation_value = random.gauss(0, self.mutation_sigma)
                        mutated_individual[i] += mutation_value
                        mutated_individual[i] = np.clip(mutated_individual[i], self.min_val, self.max_val)
                mutated_population.append(mutated_individual)

        return mutated_population

    def evolve(self) -> Tuple[List[float], float]:

        population_fitness = []
        valid_population = []
        for ind in self.population:
            if len(ind) == self.dimension:
                 fitness = self.evaluate_fitness(ind)
                 if np.isfinite(fitness):
                      population_fitness.append((ind, fitness))
                      valid_population.append(ind)

        if not valid_population:
            self.population = self._initialize_population()
            fitness_values = [self.evaluate_fitness(ind) for ind in self.population]
            best_idx = np.argmin(fitness_values) if fitness_values else 0
            return self.population[best_idx] if self.population else [], fitness_values[best_idx] if fitness_values else float('inf')

        valid_population_fitness = [(ind, fit) for ind, fit in population_fitness]
        elite_individual, elite_fitness = min(valid_population_fitness, key=lambda item: item[1])

        original_pop = self.population
        self.population = valid_population
        selected = self._tournament_selection()
        self.population = original_pop

        offspring = self._crossover(selected)
        mutated_offspring = self._mutate(offspring)

        new_population_fitness = []
        valid_new_individuals = []
        for ind in mutated_offspring:
            if len(ind) == self.dimension:
                 fitness = self.evaluate_fitness(ind)
                 if np.isfinite(fitness):
                      new_population_fitness.append((ind, fitness))
                      valid_new_individuals.append(ind)

        combined_valid_population = [elite_individual] + valid_new_individuals
        combined_valid_fitness = [elite_fitness] + [fit for _, fit in new_population_fitness]

        sorted_indices = np.argsort(combined_valid_fitness)
        next_generation = [combined_valid_population[i] for i in sorted_indices]

        needed = self.population_size - len(next_generation)
        if needed > 0:
             fillers = [ind for ind, fit in sorted(valid_population_fitness, key=lambda item: item[1]) if ind != elite_individual]
             next_generation.extend(fillers[:needed])
             needed = self.population_size - len(next_generation)
             while needed > 0:
                  next_generation.append([random.uniform(self.min_val, self.max_val) for _ in range(self.dimension)])
                  needed -= 1

        self.population = next_generation[:self.population_size]

        if not self.population:
             self.population = self._initialize_population()
             if not self.population: return [], float('inf')

        best_new_fitness = self.evaluate_fitness(self.population[0])
        return self.population[0], best_new_fitness



def run_benchmark_for_representation(representation: str, instance_name: str, instance_data: GNBG, run_id: int) -> Dict[str, Any]:
    dimension = instance_data.Dimension
    value_range = (instance_data.MinCoordinate, instance_data.MaxCoordinate)
    optimum_value = instance_data.OptimumValue
    max_fes = instance_data.MaxEvals
    acceptance_threshold = instance_data.AcceptanceThreshold

    if dimension <= 0 or max_fes <= 0:
        return {
            'run_id': run_id, 'instance': instance_name, 'representation': representation,
            'dimension': dimension, 'final_error': float('inf'), 'fes_to_threshold': max_fes,
            'success': False, 'best_fitness_found': float('inf'), 'error': 'Invalid instance parameters'
        }

    ga = None
    try:
        if representation == 'real-value':
            ga = RealValueGeneticAlgorithm(population_size=POPULATION_SIZE,
                                           dimension=dimension,
                                           value_range=value_range,
                                           mutation_rate=MUTATION_RATE,
                                           crossover_rate=CROSSOVER_RATE,
                                           selection_pressure=SELECTION_PRESSURE)
        elif representation in ['fixed-point', 'gray', 'ieee754']:
            bits_per_dim = BITS_PER_DIM_BINARY if representation != 'ieee754' else 32
            ga = BinaryGeneticAlgorithm(representation=representation,
                                        bits_per_dim=bits_per_dim,
                                        population_size=POPULATION_SIZE,
                                        dimension=dimension,
                                        value_range=value_range,
                                        mutation_rate=MUTATION_RATE,
                                        crossover_rate=CROSSOVER_RATE,
                                        selection_pressure=SELECTION_PRESSURE)
        else:
            raise ValueError(f"Neznámá reprezentace: {representation}")

        ga.set_fitness_function(instance_data)

    except Exception as e:
         return {
            'run_id': run_id, 'instance': instance_name, 'representation': representation,
            'dimension': dimension, 'final_error': float('inf'), 'fes_to_threshold': max_fes,
            'success': False, 'best_fitness_found': float('inf'), 'error': f'GA Init failed: {e}'
         }


    fes_counter = 0
    best_fitness_so_far = float('inf')
    best_individual = None
    fes_at_threshold = max_fes
    success = False
    run_error = None

    try:

        while fes_counter < max_fes:
             fes_counter += POPULATION_SIZE

             current_best_individual, current_best_fitness = ga.evolve()

             if current_best_fitness < best_fitness_so_far:
                 best_fitness_so_far = current_best_fitness
                 best_individual = list(current_best_individual) if isinstance(current_best_individual, list) else current_best_individual[:]

                 if not success and abs(best_fitness_so_far - optimum_value) < acceptance_threshold:
                      fes_at_threshold = min(fes_counter, max_fes)
                      success = True



    except KeyboardInterrupt:
         print("Běh přerušen uživatelem.")
         run_error = "Interrupted"
    except Exception as e:
        run_error = f"Evolve failed: {e}"


    final_error = float('inf')
    final_best_fitness = float('inf')
    if best_individual is not None and ga is not None:
        try:
             valid_len = False
             if isinstance(ga, BinaryGeneticAlgorithm) and len(best_individual) == ga.chromosome_length: valid_len = True
             if isinstance(ga, RealValueGeneticAlgorithm) and len(best_individual) == ga.dimension: valid_len = True

             if valid_len:
                  final_fitness = ga.evaluate_fitness(best_individual)
                  if np.isfinite(final_fitness):
                       final_error = abs(final_fitness - optimum_value)
                       final_best_fitness = final_fitness
                       if not success and final_error < acceptance_threshold:
                            success = True
                            fes_at_threshold = max_fes


        except Exception as e:
             if run_error is None: run_error = f"Final eval failed: {e}"

    report_best_fitness = final_best_fitness if np.isfinite(final_best_fitness) else best_fitness_so_far


    result_dict = {
        'run_id': run_id,
        'instance': instance_name,
        'representation': representation,
        'dimension': dimension,
        'final_error': final_error,
        'fes_to_threshold': fes_at_threshold,
        'success': success,
        'best_fitness_found': report_best_fitness,
        'max_fes_used': min(fes_counter, max_fes)
    }
    if run_error:
        result_dict['error'] = run_error
    return result_dict



Následující buňka obsahuje spouštění experimentu. Pro každou z reprezentací jsem ji spouštěl v jiném jupyter notebooku. V tomto je příklad pro real-value. Další reprezentace jsou: fixed-point, gray, ieee754.


In [11]:
if __name__ == "__main__":

    selected_representation = 'real-value'

    representations_to_run = [selected_representation]
    print(f"--- Spouštím pro reprezentaci: {selected_representation} ---")

    gnbg_instances_dict = {}
    folder_path = os.getcwd()
    print(f"Načítám GNBG instance z adresáře: {folder_path}")
    loaded_count = 0
    for i in range(1, 25):
        try:
            instance = load_instance(i, folder_path)
            if instance:
                gnbg_instances_dict[f"f{i}"] = instance
                loaded_count += 1
        except FileNotFoundError:
             print(f"Varování: Soubor f{i}.mat nebyl nalezen.")
        except Exception as e:
             print(f"Chyba při zpracování instance f{i}: {e}")

    # Mock instance
    if loaded_count == 0 :
         print("\nCHYBA: Nepodařilo se načíst žádné GNBG instance.")
         if loaded_count == 0 :
             print("Žádné instance k dispozici, končím.")
             exit()
    print(f"Celkem načteno {loaded_count} GNBG instancí.")

    results_filename = f"gnbg_results_{selected_representation}.pkl"

    all_results = []

    completed_runs_set = set()

    rerun_runs_set = set()


    if os.path.exists(results_filename):
        try:
            print(f"Nalezen soubor s výsledky: {results_filename}. Načítám...")
            with open(results_filename, "rb") as f:
                all_results_loaded = pickle.load(f)

            for res in all_results_loaded:
                if 'instance' in res and 'run_id' in res:
                    run_id_tuple = (res['instance'], res['run_id'])
                    if res.get('error') is None or res.get('error') != 'Interrupted':
                         completed_runs_set.add(run_id_tuple)
                         all_results.append(res)
                    else:
                         rerun_runs_set.add(run_id_tuple)

            print(f"Načteno {len(all_results_loaded)} předchozích záznamů.")
            print(f"Nalezeno {len(completed_runs_set)} unikátních dokončených běhů (bez 'Interrupted').")
            print(f"Nalezeno {len(rerun_runs_set)} běhů označených k opakování.")

        except Exception as e:
            print(f"Chyba při načítání nebo zpracování předchozích výsledků z {results_filename}: {e}")
            print("Začínám znovu s prázdným seznamem výsledků a množinami.")
            all_results = []
            completed_runs_set = set()
            rerun_runs_set = set()
    else:
        print(f"Soubor {results_filename} nenalezen. Začínám nový běh.")

    sorted_instance_names = sorted(gnbg_instances_dict.keys())
    total_runs_to_do = len(sorted_instance_names) * RUNS 
    completed_runs_counter_display = len(completed_runs_set)
    run_errors = 0 

    print(f"\nSpouštění benchmarku pro '{selected_representation}'. Celkem plánováno: {total_runs_to_do}. Již úspěšně dokončeno (při načtení): {len(completed_runs_set)}. K opakování: {len(rerun_runs_set)}.")
    overall_start_time = time.time()

    try:
        for instance_name in sorted_instance_names:
            instance_data = gnbg_instances_dict[instance_name]
            print(f"\nZpracovávám instanci: {instance_name} (D={instance_data.Dimension}, MaxFES={instance_data.MaxEvals})")
            instance_runs_skipped = 0
            instance_runs_executed = 0
            instance_errors_count = 0

            for rep in representations_to_run:
                for run in range(RUNS):
                    run_id_tuple = (instance_name, run)

                    if run_id_tuple in completed_runs_set:
                        instance_runs_skipped += 1
                        if instance_runs_skipped == 1:
                            print(f"\r  INFO: Některé běhy pro tuto instanci již byly úspěšně dokončeny a budou přeskočeny.", end="")
                        continue

                    instance_runs_executed += 1
                    run_start_time = time.time()

                    current_progress_count = len(completed_runs_set) + instance_runs_executed
                    progress_pct_display = ((current_progress_count -1) / total_runs_to_do) * 100 if total_runs_to_do > 0 else 0


                    print(f"\r  {rep} - Běh {run+1}/{RUNS} (probíhá) [{progress_pct_display:.1f}%]...", end="")

                    result = run_benchmark_for_representation(rep, instance_name, instance_data, run)

                    run_duration = time.time() - run_start_time

                    result['run_duration_seconds'] = run_duration

                    all_results = [r for r in all_results if (r.get('instance'), r.get('run_id')) != run_id_tuple]
                    all_results.append(result) 
                    error_msg = result.get('error', None)
                    if result.get('error') != 'Interrupted':
                        completed_runs_set.add(run_id_tuple)
                        if run_id_tuple in rerun_runs_set:
                             rerun_runs_set.remove(run_id_tuple)

                    status = f"OK (Err={result.get('final_error', float('inf')):.2e}, Succ={result.get('success', False)})"
                    if error_msg:
                         run_errors += 1
                         instance_errors_count += 1
                         status = f"CHYBA: {error_msg}"

                    progress_pct_display = (len(completed_runs_set) / total_runs_to_do) * 100 if total_runs_to_do > 0 else 0
                    print(f"\r  {rep} - Běh {run+1}/{RUNS} (proveden) ({run_duration:.1f}s) [{progress_pct_display:.1f}%] - {status}   ", end="")



                    try:
                        with open(results_filename, "wb") as f:
                            pickle.dump(all_results, f)
                    except Exception as e:
                        print(f"\nCHYBA při průběžném ukládání do {results_filename} po {run_id_tuple}: {e}")

                print() 
                if instance_runs_skipped > 0:
                    print(f"  Instance {instance_name}: Přeskočeno {instance_runs_skipped} již úspěšně dokončených běhů.")
                if instance_runs_executed > 0:
                    print(f"  Instance {instance_name}: Provedeno {instance_runs_executed} nových nebo opakovaných běhů v této session.")
                if instance_errors_count > 0:
                    print(f"  Instance {instance_name}: Zaznamenáno {instance_errors_count} chyb v nových/opakovaných bězích v této session.")
                instance_all_done = all((instance_name, r) in completed_runs_set for r in range(RUNS))
                if instance_all_done:
                    print(f"  Instance {instance_name}: Všechny naplánované běhy ({RUNS}) jsou nyní úspěšně dokončeny.")
                else:
                    remaining_instance_runs = [(instance_name, r) for r in range(RUNS) if (instance_name, r) not in completed_runs_set]
                    print(f"  Instance {instance_name}: Zbývá úspěšně dokončit {len(remaining_instance_runs)} běhů.")


    except KeyboardInterrupt:
        print("\n!!! BĚH PŘERUŠEN UŽIVATELEM (KeyboardInterrupt) !!!")
    except Exception as e:
        print(f"\n!!! NEOČEKÁVANÁ CHYBA BĚHEM ZPRACOVÁNÍ: {e} !!!")
    finally:
        overall_duration = time.time() - overall_start_time
        print(f"\nBenchmark pro '{selected_representation}' (nebo jeho část) běžel {overall_duration:.2f}s v této session.")
        if run_errors > 0:
             print(f"POZOR: Během této session došlo k {run_errors} chybám v nově provedených bězích.")
        remaining_runs_count = total_runs_to_do - len(completed_runs_set)
        if remaining_runs_count > 0:
             print(f"Zbývá dokončit {remaining_runs_count} běhů (nebyly dosud úspěšně dokončeny).")
        else:
             print("Všechny naplánované běhy byly úspěšně dokončeny.")

        print(f"Pokus o finální uložení {len(all_results)} celkových záznamů výsledků do {results_filename}...")
        try:
            with open(results_filename, "wb") as f:
                pickle.dump(all_results, f)
            print(f"Výsledky úspěšně uloženy/aktualizovány v {results_filename}.")
        except Exception as e:
            print(f"CHYBA při finálním ukládání do {results_filename}: {e}")

        print(f"\nSkript pro reprezentaci '{selected_representation}' dokončen (nebo přerušen).")


--- Spouštím pro reprezentaci: real-value ---
Načítám GNBG instance z adresáře: d:\2025skola\EV_ALG_DIP\GNBG_Instances_Python
Celkem načteno 24 GNBG instancí.
Nalezen soubor s výsledky: gnbg_results_real-value.pkl. Načítám...
Načteno 744 předchozích záznamů.
Nalezeno 744 unikátních dokončených běhů (bez 'Interrupted').
Nalezeno 0 běhů označených k opakování.

Spouštění benchmarku pro 'real-value'. Celkem plánováno: 744. Již úspěšně dokončeno (při načtení): 744. K opakování: 0.

Zpracovávám instanci: f1 (D=30, MaxFES=500000)
  INFO: Některé běhy pro tuto instanci již byly úspěšně dokončeny a budou přeskočeny.
  Instance f1: Přeskočeno 31 již úspěšně dokončených běhů.
  Instance f1: Všechny naplánované běhy (31) jsou nyní úspěšně dokončeny.

Zpracovávám instanci: f10 (D=30, MaxFES=500000)
  INFO: Některé běhy pro tuto instanci již byly úspěšně dokončeny a budou přeskočeny.
  Instance f10: Přeskočeno 31 již úspěšně dokončených běhů.
  Instance f10: Všechny naplánované běhy (31) jsou nyní 